In [ ]:
#####Gaussian Process Spatio Temporal point model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.matlib
import scipy.linalg
import itertools
import calendar
from datetime import datetime
from datetime import timedelta
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [ ]:
##乱数を生成する関数を設定
#多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

#多変量正規分布の乱数を生成する関数
def rmvnorm(mu, Cov, hh, k):
    s = mu + np.random.normal(0, 1, hh*k).reshape(hh, k)
    P = np.linalg.cholesky(Cov)
    y = np.dot(P, s.T).T
    return y

#任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper, pattern):
    #相関行列の初期値を定義する
    if pattern==1:
        Prob = np.abs(lower) / (np.abs(lower) + np.abs(upper))
        z = np.random.binomial(1, Prob, col*col); z[z==0] = -1
        cov_vec = np.random.beta(1.0, 3.0, col*col) * z   #相関係数の乱数ベクトルを作成
    else:
        cov_vec = np.random.uniform(lower, upper, col*col)   #相関係数の乱数ベクトルを作成
    rho = np.tril(cov_vec.reshape(col, col), k=-1)   #乱数ベクトルを下三角行列化
    Sigma = (rho + rho.T) + np.diag(np.repeat(1, col))   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = np.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(col):
        if eigen_val[i] < 0:
            eigen_val[i] = np.random.uniform(eigen_lower, eigen_upper, 1)
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(np.power(np.diag(Sigma), 0.5)[:, np.newaxis], np.power(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

#相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

#分散共分散行列から相関行列に変換する関数
def cov2cor(Cov):
    D = np.diag(np.power(np.diag(Cov), -1/2))
    corr = np.dot(np.dot(D, Cov), D)
    return corr

In [ ]:
####データの生成####
##データの設定
models = 2
weeks = 7
k = 10
place = 3000
category = 150
max_m = 5
Lambda = np.random.gamma(75.0, 1/0.5, place)
pt = np.random.poisson(Lambda, place)
N = np.sum(pt)
m = np.random.poisson(1.75, N)
m[m==0] = 1; m[m > max_m] = max_m 
m_vec = np.repeat(1.0, max_m)
k_vec = np.repeat(1.0, k)

In [ ]:
##idとインデックスを作成
#idの作成
d_id = np.repeat(np.arange(place), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(place)])))

#インデックスの設定
d_list = [i for i in range(place)]
for i in range(place):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [ ]:
np.random.multinomial(m[i], phi[point[i], ], 1)


In [ ]:
##カテゴリー割当を定義
#トピックを生成
s = 30
prob = np.random.dirichlet(np.repeat(0.75, s), 1).reshape(-1)
theta = np.random.dirichlet(np.repeat(0.2, s), place)
phi = np.random.dirichlet(np.repeat(0.1, category), s)
point = rmnom(theta[d_id, ], N, s, 1)[0]
n = np.array([np.sum(point==j) for j in range(s)])

#多項分布からカテゴリーを生成
category_vec = np.arange(category)
category_id = np.array(np.full((N, max_m), category), dtype="int16")
for i in range(N):
    while True:
        generate = np.random.multinomial(m[i], phi[point[i], ], 1)
        if np.max(generate)==1:
            new_generate = generate * category_vec
            category_id[i, np.arange(m[i])] = new_generate[generate > 0]
            break
            
#インデックスを定義
category_list = [i for i in range(category)]
for i in range(category):
    category_list[i] = np.array(np.where(category_id==i)[0], dtype="int")

In [ ]:
##観測地点間の距離分布を生成
#場所集合のトピックを生成
s = 30
prob = np.random.dirichlet(np.repeat(1.0, s), 1).reshape(-1)
point = np.dot(np.random.multinomial(1, prob, place), np.arange(s))
n = np.array([np.sum(point==j) for j in range(s)])

#観測地点の経緯度を生成
longitude = np.array([0, 5]); latitude = np.array([0, 5])
geo_spot0 = np.zeros((place, 2))
for j in range(s):
    index = np.array(np.where(point==j)[0], dtype="int")
    mu = np.append(np.random.uniform(longitude[0], longitude[1]), np.random.uniform(latitude[0], latitude[1]))
    Cov = np.diag(np.random.uniform(0.01, 0.15, 2))
    cor = np.random.uniform(-0.6, 0.6, 1) * np.prod(np.sqrt(np.diag(Cov)))
    Cov[0, 1] = cor; Cov[1, 0] = cor
    geo_spot0[index, ] = np.random.multivariate_normal(mu, Cov, n[j])
geo_spot = np.min(geo_spot0) + geo_spot0

#経緯度を可視化
fig = plt.figure(figsize=(10.0, 5.75))
plt.scatter(geo_spot[:, 0], geo_spot[:, 1], alpha=0.5, linewidths="2", s=4)
plt.title("観測地点の分布")
plt.xlabel("経度", fontsize=12.0)
plt.ylabel("緯度", fontsize=12.0)

In [ ]:
#場所間のユークリッド距離を定義
d = np.zeros((place, place))
for i in range(place):
    d[i, ] = np.sqrt(np.sum(np.power(geo_spot[np.repeat(i, place), ] - geo_spot, 2), axis=1))

#分布を可視化
fig = plt.figure(figsize=(10.0, 5.75))
plt.hist(d[np.triu(d, k=1)!=0.0], bins=25)
plt.title("ユークリッド距離の分布")
plt.xlabel("ユークリッド距離", fontsize=12.0)
plt.ylabel("頻度", fontsize=12.0)

In [ ]:
##応答変数の生成
#分散共分散行列を生成
I = np.diag(np.repeat(1.0, place))
phi1 = np.array([1.25]); phi2 = np.array([1.0])
omega1 = np.array([0.75]); omega2 = np.array([0.5])
gamma1 = np.array([0.7]); gamma2 = np.array([0.5])
delta1 = np.array([phi1, omega1, gamma1]).reshape(-1)
delta2 = np.array([phi2, omega2, gamma2]).reshape(-1)
Cov1 = omega1*I + gamma1*np.exp(-phi1 * d)
Cov2 = omega2*I + gamma2*np.exp(-phi2 * d)
deltat1 = delta1.copy(); deltat2 = delta2.copy()
Covt1 = Cov1.copy(); Covt2 = Cov2.copy()

#事前分布を定義
alpha1 = 0.0
alpha2 = 0.0
alpha_u1 = np.repeat(0.0, place)
alpha_u2 = np.repeat(0.0, place)
tau1 = np.array([1.0])
tau2 = np.array([0.75])
kappa1 = np.random.uniform(3.5, 7.0, 1)
kappa2 = np.random.uniform(3.5, 7.0, 1)
taut = np.append(tau1, tau2)
kappat = np.append(kappa1, kappa2)

#モデルパラメータを生成
beta1 = np.array([0.75])
beta2 = np.array([0.85])
beta_u1 = np.append(np.random.normal(alpha1, tau1, category), 0.0)
beta_u2 = np.append(np.random.normal(alpha2, tau2, category), 0.0)
theta_u1 = np.random.multivariate_normal(alpha_u1, Cov1, 1).reshape(-1)
theta_u2 = np.random.multivariate_normal(alpha_u2, Cov2, 1).reshape(-1)
betat = np.append(beta1, beta2); betat_u1 = beta_u1.copy(); betat_u2 = beta_u2.copy()
thetat_u1 = theta_u1.copy(); thetat_u2 = theta_u2.copy()

#フォンミーゼス分布から円周データを生成
arc_mu1 = 2*np.arctan(beta1 + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec))
arc_mu2 = 2*np.arctan(beta2 + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec))
y1 = np.random.vonmises(arc_mu1, kappa1, N)
y2 = np.random.vonmises(arc_mu2, kappa2, N)

In [ ]:
#生成したデータを可視化
dt = [arc_mu1, arc_mu2, y1, y2]
fig_range = np.append(np.array([np.min(arc_mu1), np.min(arc_mu2), np.min(y1), np.min(y2)]), 
                      np.array([np.max(arc_mu1), np.max(arc_mu2), np.max(y1), np.max(y2)])).reshape(2, len(dt))
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
legend = ["1日周期の期待値の分布", "週周期の期待値の分布", "1日周期の応答変数の分布", "週周期の応答変数の分布"]
fig = plt.figure(figsize=(12.0, 7.0))
for j in range(len(dt)):
    ax = fig.add_subplot(2, 2, j+1)
    ax.hist(dt[j],  bins=25, range=(fig_range[0, j], fig_range[1, j]), color=colorlist[j])
    plt.title(legend[j], fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
####ハミルトニアンモンテカルロ法でパラメータを推定####
##GP Spatio Temporal point modelを推定するための関数
#多変量正規分布の条件付き期待値と分散を計算する関数
def cdMVN(mu, Cov, department, U):

    #分散共分散行列のブロック行列を定義
    index = np.delete(np.arange(Cov.shape[0]), department)
    Cov11 = Cov[department, ][:, department]
    Cov12 = Cov[department, ][:, index]
    Cov21 = Cov[:, department][index, ]
    Cov22 = Cov[index, ][:, index]

    #条件付き分散と条件付き平均を計算
    inv_Cov22 = np.linalg.inv(Cov22)
    CDinv = np.dot(Cov12, inv_Cov22)
    CDmu = mu[:, department] + np.dot(CDinv, (U[:, index] - mu[:, index]).T).T   #条件付き平均
    CDvar = Cov11 - np.dot(np.dot(Cov12, inv_Cov22), Cov21)   #条件付き分散
    return CDmu, CDvar

#Von Mises Regression modelの対数尤度関数
def loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec):
    
    #モデルの期待値
    arc_mu1 = 2*np.arctan(beta[0] + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec))
    arc_mu2 = 2*np.arctan(beta[1] + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec))
    
    #対数尤度の和を計算
    LL1 = -np.log(2*np.pi*scipy.special.i0(kappa[0])) + kappa[0]*np.cos(y1 - arc_mu1)
    LL2 = -np.log(2*np.pi*scipy.special.i0(kappa[1])) + kappa[1]*np.cos(y2 - arc_mu2)
    LL = np.hstack((LL1[:, np.newaxis], LL2[:, np.newaxis]))
    return LL

In [ ]:
##期待値パラメータをサンプリングするための関数
#期待値パラメータの対数事後分布    
def Posterior_mu(beta, inv_tau):
    LL = np.sum(loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec), axis=0)
    Prior = -1/2 * np.dot(np.dot(beta, inv_tau), beta)
    Posterior = LL + Prior
    return Posterior

#期待値パラメータの対数事後分布の勾配ベクトル
def dll_mu(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, m_vec):
    #モデルの期待値
    mu1 = beta[0] + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec)
    mu2 = beta[1] + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec)
    arc_mu1 = 2*np.arctan(mu1)
    arc_mu2 = 2*np.arctan(mu2)

    #対数尤度の勾配ベクトル
    LLd_sum1 = np.sum(kappa[0] * (np.sin(y1 - arc_mu1)) * (2 * (1 / (1 + np.power(mu1, 2)))))
    LLd_sum2 = np.sum(kappa[1] * (np.sin(y2 - arc_mu2)) * (2 * (1 / (1 + np.power(mu2, 2)))))
    LLd_sum = np.append(LLd_sum1, LLd_sum2)

    #事前分布の勾配ベクトル
    dmvn = -np.dot(inv_tau, beta)
    gradient = -(LLd_sum + dmvn)
    return gradient

#期待値パラメータのリープフロッグ法を解く関数
def leapfrog_mu(r, z, D, e, L): 
    def leapfrog_step(r, z, e):
        r2 = r - e * D(z, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, m_vec) / 2
        z2 = z + e * r2
        r2 = r2 - e * D(z2, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, m_vec) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [ ]:
#カテゴリーパラメータの対数事後分布
def Posterior_category(beta_u1, beta_u2, inv_tau_u, category, category_list, models):
    #カテゴリーごとの対数尤度の和
    beta_u = np.hstack((beta_u1[:, np.newaxis], beta_u2[:, np.newaxis]))
    LLi = loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec)
    LL = np.zeros((category, models))
    for i in range(category):
         LL[i] = np.sum(LLi[category_list[i], ], axis=0)
            
    #対数事後分布の和
    Prior = -1/2 * (beta_u[:category, ] * np.diag(inv_tau_u) * beta_u[:category, ])
    Posterior = LL + Prior
    return Posterior

#カテゴリーパラメータの対数事後分布の勾配ベクトル
def dll_category(beta, beta_u, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, category_list, models, m_vec):
    #モデルの期待値
    mu1 = beta[0] + theta_u1[d_id] + np.dot(np.append(beta_u[:, 0], 0)[category_id], m_vec)
    mu2 = beta[1] + theta_u2[d_id] + np.dot(np.append(beta_u[:, 1], 0)[category_id], m_vec)
    arc_mu1 = 2*np.arctan(mu1)
    arc_mu2 = 2*np.arctan(mu2)

    #対数尤度の勾配ベクトル
    LLd1 = kappa[0] * (np.sin(y1 - arc_mu1)) * (2 * (1 / (1 + np.power(mu1, 2))))
    LLd2 = kappa[1] * (np.sin(y2 - arc_mu2)) * (2 * (1 / (1 + np.power(mu2, 2))))
    LLd_sums = np.zeros((category, models))
    for i in range(category):
        LLd_sums[i, ] = np.append(np.sum(LLd1[category_list[i]]), np.sum(LLd2[category_list[i]]))

    #事前分布の勾配ベクトル
    dmvn = -np.dot(inv_tau_u, beta_u.T).T
    gradient = -(LLd_sums + dmvn)
    return gradient

#カテゴリーパラメータのリープフロッグ法を解く関数
def leapfrog_category(r, z, D, e, L): 
    def leapfrog_step(r, z, e):
        r2 = r - e * D(beta, z, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, category_list, models, m_vec) / 2
        z2 = z + e * r2
        r2 = r2 - e * D(beta, z2, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, category_list, models, m_vec) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [ ]:
##ガウス過程のパラメータをサンプリングするための関数
#ガウス過程のパラメータの対数事後分布
def Posterior_gp(theta_u1, theta_u2, inv_Cov1, inv_Cov2):
    LL = np.sum(loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec), axis=0)
    Prior1 = -1/2 * np.dot(np.dot(theta_u1, inv_Cov1), theta_u1)
    Prior2 = -1/2 * np.dot(np.dot(theta_u2, inv_Cov2), theta_u2)
    Posterior1 = LL[0] + Prior1
    Posterior2 = LL[1] + Prior2
    Posterior = np.append(Posterior1, Posterior2)
    return Posterior

#ガウス過程のパラメータの対数事後分布の勾配ベクトル
def dll_gp(beta, beta_u1, beta_u2, theta_u, kappa, y1, y2, inv_Cov1, inv_Cov2, place, d_id, category_id, d_list, models, m_vec):
    #モデルの期待値
    mu1 = beta[0] + theta_u[d_id, 0] + np.dot(beta_u1[category_id, ], m_vec)
    mu2 = beta[1] + theta_u[d_id, 1] + np.dot(beta_u2[category_id, ], m_vec)
    arc_mu1 = 2*np.arctan(mu1)
    arc_mu2 = 2*np.arctan(mu2)

    #対数尤度の勾配ベクトル
    LLd1 = kappa[0] * (np.sin(y1 - arc_mu1)) * (2 * (1 / (1 + np.power(mu1, 2))))
    LLd2 = kappa[1] * (np.sin(y2 - arc_mu2)) * (2 * (1 / (1 + np.power(mu2, 2))))
    LLd_sums = np.zeros((place, models))
    for i in range(place):
        LLd_sums[i, ] = np.append(np.sum(LLd1[d_list[i]]), np.sum(LLd2[d_list[i]]))

    #ガウス過程事前分布の勾配ベクトル
    dmvn1 = -np.dot(inv_Cov1, theta_u[:, 0])
    dmvn2 = -np.dot(inv_Cov2, theta_u[:, 1])
    gradient1 = -(LLd_sums[:, 0] + dmvn1)
    gradient2 = -(LLd_sums[:, 1] + dmvn2)
    gradient = np.hstack((gradient1[:, np.newaxis], gradient2[:, np.newaxis]))
    return gradient

#ガウス過程のパラメータのリープフロッグ法を解く関数
def leapfrog_gp(r, z, D, e, L): 
    def leapfrog_step(r, z, e):
        r2 = r - e * D(beta, beta_u1, beta_u2, z, kappa, y1, y2, inv_Cov1, inv_Cov2,
                       place, d_id, category_id, d_list, models, m_vec) / 2
        z2 = z + e * r2
        r2 = r2 - e * D(beta, beta_u1, beta_u2, z2, kappa, y1, y2, inv_Cov1, inv_Cov2,
                        place, d_id, category_id, d_list, models, m_vec) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [ ]:
##分散パラメータを推定するための関数
#分散パラメータの推定用の対数尤度関数
def Lho(kappa, mu, y):
    LL = -np.log(2*np.pi*scipy.special.i0(kappa)) + kappa*np.cos(y - mu)
    return LL 

#分散パラメータの勾配ベクトル
def dll_kappa(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, d_id, category_id, m_vec):

    #モデルの期待値
    mu1 = beta[0] + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec)
    mu2 = beta[1] + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec)
    arc_mu1 = 2*np.arctan(mu1)
    arc_mu2 = 2*np.arctan(mu2)

    #分散パラメータの対数尤度の勾配
    h = 1e-5
    LLd1 = (Lho(kappa[0]+h, arc_mu1, y1) - Lho(kappa[0]-h, arc_mu1, y1)) / (2*h)
    LLd2 = (Lho(kappa[1]+h, arc_mu2, y2) - Lho(kappa[1]-h, arc_mu2, y2)) / (2*h)
    LLd = np.hstack((LLd1[:, np.newaxis], LLd2[:, np.newaxis]))
    return LLd


#分散共分散行列のパラメータの勾配ベクトル
def dll_delta(theta, Cov, inv_Cov, er, d):
    
    #パラメータを定義
    phi = theta[0]
    tau = theta[1]
    Sigma = theta[2]
    
    #勾配ベクトルを定義
    d_expon1 = -(Sigma * (np.exp(-phi * d) * d))
    d_expon2 = np.exp(-phi * d)
    dll1 = -1/2 * (np.sum(np.diag(np.dot(inv_Cov, d_expon1))) - np.dot(np.dot(np.dot(np.dot(er, inv_Cov), d_expon1), inv_Cov), er))
    dll2 = -1/2 * (np.sum(np.diag(inv_Cov)) - np.dot(np.dot(np.dot(er, inv_Cov), inv_Cov), er))
    dll3 = -1/2 * (np.sum(np.diag(np.dot(inv_Cov, d_expon2))) - np.dot(np.dot(np.dot(np.dot(er, inv_Cov), d_expon2), inv_Cov), er))
    dll = np.array([dll1, dll2, dll3])
    return dll

In [ ]:
##アルゴリズムの設定
k = 10
R = 2000
keep = 2
burnin = int(500/keep)
iter = 0
disp = 10
e1 = 0.001
e2 = 0.0025
e3 = 0.01
eta1 = 0.5
eta2 = 0.25
L = 3

In [ ]:
#ミニバッチを設定
size = 25   #ミニバッチ数
sample = np.array(N/m, dtype="int")   #ミニバッチのサンプル数
u = np.array(np.argsort(-np.random.uniform(0, 1, N)), dtype="int")
index = np.array_split(np.arange(N), size)
index_sample = [i for i in range(size)]
for i in range(size):
    index_sample[i] = u[index[i]]

In [ ]:
#事前分布の設定
tau = np.diag(np.repeat(100.0, models))
inv_tau = np.linalg.inv(tau)
omega1 = deltat1[0]
omega2 = deltat2[0]
s0 = 0.25
v0 = 0.25

In [ ]:
##パラメータの真値
#階層モデルの事前分布の真値
tau_u = np.diag(taut.copy())
inv_tau_u = np.linalg.inv(tau_u)
delta1 = deltat1.copy()
delta2 = deltat2.copy()
index_delta = np.array([0, 2])

#モデルパラメータの真値
beta = betat.copy()
beta_u1 = betat_u1.copy()
beta_u2 = betat_u2.copy()
theta_u1 = thetat_u1.copy()
theta_u2 = thetat_u2.copy()
kappa = kappat.copy()
Cov1 = Covt1.copy()
Cov2 = Covt2.copy()
inv_Cov1 = np.linalg.inv(Cov1)
inv_Cov2 = np.linalg.inv(Cov2)

#期待値の真値
arc_mu1 = 2*np.arctan(beta1 + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec))
arc_mu2 = 2*np.arctan(beta2 + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec))

In [ ]:
##パラメータの初期値
#階層モデルの事前分布の初期値
tau_u = np.diag(np.array([2.5, 2.5]))
inv_tau_u = np.linalg.inv(tau_u)
delta1 = np.array([1.0, 1.0, 1.0])
delta2 = np.array([1.0, 1.0, 1.0])

#モデルパラメータの初期値
beta = np.random.uniform(0.5, 1.5, models)
beta_u1 = np.append(np.random.normal(0, 1.0, category), 0.0)
beta_u2 = np.append(np.random.normal(0, 1.0, category), 0.0)
theta_u1 = np.random.normal(0, 0.5, place)
theta_u2 = np.random.normal(0, 0.5, place)
kappa = np.repeat(5.0, models)
Cov1 = delta1[1]*I + delta1[2]*np.exp(-delta1[0] * d)
Cov2 = delta2[1]*I + delta2[2]*np.exp(-delta2[0] * d)
inv_Cov1 = np.linalg.inv(Cov1)
inv_Cov2 = np.linalg.inv(Cov2)

#期待値の初期値
arc_mu1 = 2*np.arctan(beta1 + theta_u1[d_id] + np.dot(beta_u1[category_id, ], m_vec))
arc_mu2 = 2*np.arctan(beta2 + theta_u2[d_id] + np.dot(beta_u2[category_id, ], m_vec))

In [ ]:
##サンプリング結果の格納用配列
#階層モデルのパラメータの格納用配列
TAU_U = np.zeros((int(R/keep), models))
DELTA1 = np.zeros((int(R/keep), delta1.shape[0]))
DELTA2 = np.zeros((int(R/keep), delta2.shape[0]))

#モデルパラメータの格納用配列
BETA = np.zeros((int(R/keep), models))
BETA_U = np.zeros((category+1, models, int(R/keep)))
THETA_U = np.zeros((place, models, int(R/keep)))
KAPPA = np.zeros((int(R/keep), models))

In [ ]:
##対数尤度の基準値
#初期値での対数尤度
LLst = np.sum(loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec), axis=0)
LLst = np.append(LLst, np.sum(LLst))
print(np.round(LLst, 1))

#真値での対数尤度
LLbest = np.sum(loglike(betat, betat_u1, betat_u2, thetat_u1, thetat_u2, kappat, y1, y2, N, d_id, category_id, m_vec), axis=0)
LLbest = np.append(LLbest, np.sum(LLbest))
print(np.round(LLbest, 1))

In [ ]:
####パラメータをサンプリング####
for rp in range(R):
    
    ##期待値パラメータをサンプリング
    #HMCの新しいパラメータを生成
    rold = np.random.normal(0, 1.0, models)
    betad = beta.copy()

    #リープフロッグ法による1ステップ移動
    res = leapfrog_mu(rold, betad, dll_mu, e1, L)
    rnew = res[0]
    betan = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -Posterior_mu(betan, inv_tau) + np.power(rnew, 2)/2
    Hold = -Posterior_mu(betad, inv_tau) + np.power(rold, 2)/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, models)
    gamma = np.min(np.vstack((np.repeat(1.0, models), np.exp(Hold - Hnew))), axis=0)
    accept_prob1 = gamma.copy()
    flag = np.array(gamma > rand, dtype="int")
    beta = flag*betan + (1-flag)*betad


    ##カテゴリーパラメータをサンプリング
    #HMCの新しいパラメータを生成
    rold = np.random.normal(0, 1.0, models*category).reshape(category, models)
    betad_u = np.vstack((np.hstack((beta_u1[:category][:, np.newaxis], beta_u2[:category][:, np.newaxis])), np.repeat(0, models)))

    #リープフロッグ法による1ステップ移動
    res = leapfrog_category(rold, betad_u[:category], dll_category, e2, L)
    rnew = res[0]
    betan_u = np.vstack((res[1], np.repeat(0, models)))

    #移動前と移動後のハミルトニアン
    Hnew = -Posterior_category(betan_u[:, 0], betan_u[:, 1], inv_tau_u, category, category_list, models) + np.power(rnew, 2)/2
    Hold = -Posterior_category(betad_u[:, 0], betad_u[:, 1], inv_tau_u, category, category_list, models) + np.power(rold, 2)/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, models*category).reshape(category, models)
    gamma = np.exp(Hold - Hnew); gamma[gamma > 1.0] = 1.0
    accept_prob2 = gamma.copy()
    flag = np.vstack((np.array(gamma > rand, dtype="int"), np.repeat(1, models)))
    beta_u = flag*betan_u + (1-flag)*betad_u
    beta_u1 = beta_u[:, 0]
    beta_u2 = beta_u[:, 1]


    ##ガウス過程のパラメータをサンプリング
    #HMCの新しいパラメータをサンプリング
    rold = np.random.normal(0, 1.0, models*place).reshape(place, models)
    thetad_u = np.hstack((theta_u1[:, np.newaxis], theta_u2[:, np.newaxis]))

    #リープフロッグ法による1ステップ移動
    res = leapfrog_gp(rold, thetad_u, dll_gp, e3, L)
    rnew = res[0]
    thetan_u = res[1]
    thetan_u1 = thetan_u[:, 0]
    thetan_u2 = thetan_u[:, 1]

    #移動前と移動後のハミルトニアン
    Hnew = -Posterior_gp(thetan_u1, thetan_u2, inv_Cov1, inv_Cov2) + np.sum(np.power(rnew, 2), axis=0)/2
    Hold = -Posterior_gp(theta_u1, theta_u2, inv_Cov1, inv_Cov2) + np.sum(np.power(rold, 2), axis=0)/2

    #新しいパラメータを採択
    rand = np.random.uniform(0, 1, models*place).reshape(place, models)
    gamma = np.exp(Hold - Hnew); gamma[gamma > 1.0] = 1.0
    accept_prob3 = gamma.copy()
    flag = np.array(gamma > rand, dtype="int")
    theta_u = flag*thetan_u + (1-flag)*thetad_u
    theta_u1 = theta_u[:, 0]
    theta_u2 = theta_u[:, 1]
    
    
    ##確率的勾配降下法で分散パラメータを更新
    #ミニバッチの勾配ベクトルの和を計算
    index = index_sample[np.argmax(np.random.multinomial(1, np.repeat(1/size, size), 1))]
    g = np.sum(dll_kappa(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, 
                         y1[index], y2[index], d_id[index], category_id[index], m_vec), axis=0)

    #パラメータを更新
    index_size = index.shape[0]
    kappa += eta1 * (g / index_size)
    
    
    ##分散共分散行列のパラメータを更新
    #勾配ベクトルからパラメータを更新
    dll1 = dll_delta(delta1, Cov1, inv_Cov1, theta_u1, d)
    dll2 = dll_delta(delta2, Cov2, inv_Cov2, theta_u2, d)
    delta1[index_delta] += eta2 * (dll1 / place)[index_delta]
    delta2[index_delta] += eta2 * (dll2 / place)[index_delta]
    
    #期待値と分散共分散行列を更新
    Cov1 = delta1[1]*I + delta1[2]*np.exp(-delta1[0] * d)
    Cov2 = delta2[1]*I + delta2[2]*np.exp(-delta2[0] * d)
    inv_Cov1 = np.linalg.inv(Cov1)
    inv_Cov2 = np.linalg.inv(Cov2)


    ##カテゴリーパラメータの分散をサンプリング
    #ガンマ分布のパラメータ
    s1 = np.sum(np.power(beta_u1 - np.mean(beta_u1), 2)) + s0
    s2 = np.sum(np.power(beta_u2 - np.mean(beta_u2), 2)) + s0
    v1 = category + v0
    v2 = category + v0

    #逆ガンマ分布から分散をサンプリング
    tau1 = 1/np.random.gamma(v1/2, 1/(s1/2), 1)
    tau2 = 1/np.random.gamma(v2/2, 1/(s2/2), 1)
    tau_u = np.diag(np.append(tau1, tau2))
    inv_tau_u = np.linalg.inv(tau_u)


    ##サンプリング結果の保存と表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = int(rp/keep)
        TAU_U[mkeep, ] = np.diag(tau_u)
        DELTA1[mkeep, ] = delta1
        DELTA2[mkeep, ] = delta2
        BETA[mkeep, ] = beta
        BETA_U[:, :, mkeep] = beta_u
        THETA_U[:, :, mkeep] = theta_u
        KAPPA[mkeep, ] = kappa
        
    if rp%disp==0:  
        #対数尤度の更新
        LL = np.sum(loglike(beta, beta_u1, beta_u2, theta_u1, theta_u2, kappa, y1, y2, N, d_id, category_id, m_vec), axis=0)
        LL = np.append(LL, np.sum(LL))

        #サンプリング結果を表示
        print(rp)
        print(np.round(np.append(LL, LLbest), 1))
        print(np.round(np.append(np.append(accept_prob1, np.mean(accept_prob2, axis=0)), np.mean(accept_prob3, axis=0)), 3))
        print(np.round(delta1, 3))
        print(np.round(delta2, 3))
        print(np.round(kappa, 3))